#### LeetCode - SQL - #626

Refer [626. Exchange Seats](https://leetcode.com/problems/exchange-seats/description/)

Write a solution to swap the seat id of every two consecutive students. If the number of students is odd, the id of the last student is not swapped.

Return the result table ordered by  `id`  **in ascending order**.

In [ ]:
data_seat = [[1, 'Abbot'], [2, 'Doris'], [3, 'Emerson'], [4, 'Green'], [5, 'Jeames']]
columns_seat = ['id', 'student']
schema_seat = {'id':'Int64', 'student':'object'}

In [ ]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName('LeetCode SQL').getOrCreate()

In [ ]:
# Create Spark Dataframe using Pandas
import pandas

pandas_df_seat = pandas.DataFrame(data=data_seat, columns=columns_seat).astype(schema_seat)

df_seat = spark_context.createDataFrame(pandas_df_seat)
df_seat.schema
df_seat.show(5)

In [ ]:
# Create Spark Dataframe using PySpark functions - 1
schema_seat = 'id integer, student string'

df_seat = spark_context.createDataFrame(data=data_seat, schema=schema_seat)
df_seat.schema
df_seat.show(5)

In [ ]:
# Create Spark Dataframe using PySpark functions - 2
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema_seat = StructType([
    StructField('id', IntegerType(), False),
    StructField('student', StringType(), True)
])

df_seat = spark_context.createDataFrame(data=data_seat, schema=schema_seat)
df_seat.schema
df_seat.show(5)

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

window_spec = Window.orderBy(df_seat.id.asc())

df_seat \
    .select(
        df_seat.id.alias('id'),
        F.when(
            ((df_seat.id % 2) == 0),
            F.lag(df_seat.student).over(window_spec)
        ).otherwise(
            F.when(
                F.lead(df_seat.student).over(window_spec).isNotNull(),
                F.lead(df_seat.student).over(window_spec)
            ).otherwise(
                df_seat.student
            )
        ).alias('student')
    ).show()